In [ ]:
# GAMMA VERSION 3.0 ===========================================================
# Keep access Token
# Call out API to perform tasks
#   - Update kill instance Chrome
# =============================================================================
# Library for process Token - using Chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for POST GET requests
import requests
import urllib.parse
import socket
import select
import json
import subprocess
import os

# Global configuration ========================================================
access_token = None
new_token = None
expires_token = None
# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')
# Check point of important handling - in UNIX timestamp
check_point = time.time()
last_order = 0

# Define the list of allowed coins to order ===================================
tuple_coin = (
  "UNIVNDC","CETUSVNDC","DOGEVNDC","JASMYVNDC","SOLVNDC",
  "MCAKEVNDC","SXPVNDC","CKBVNDC","EGPT1000VNDC","UXLINKVNDC",
  "MIAVNDC","MOODENGVNDC","EDU3VNDC","MEON1000VNDC","DOGSVNDC",
  "SUIVNDC","FOXYVNDC","BIGTIMEVNDC","TRUVNDC","OPVNDC",
  "LINKVNDC","XRPVNDC","1000CATVNDC","WIFVNDC","NEIROVNDC",
  "XLMVNDC","KSMVNDC","DOTVNDC","ENSVNDC","SANDVNDC",
  "GRASSVNDC","LYNXVNDC","AVAXVNDC","LISTAVNDC","TIAVNDC",
  "PEPE1000VNDC","WLDVNDC", "ADAVNDC", "HBARVNDC", "LDOVNDC"
)
# Define your quiet hours not to send Telegram
quiet_hours = [
    ((22, 0),(23, 59)),  # 10 PM to midnight
    ((0, 0),(6, 0))      # Midnight to 6 AM
]

# Function to send a message to Telegram
def send_telegram(message_string, max_retry=2):
    # Telegram bot token and chat ID
    TOKEN = "5614737400:AAHbvZrJbomt09EkpPuhadBCJl7NaGu6rlg"
    ID = "5559031253"

    # Check if the time now is quiet hour or not
    current_time = datetime.now(timezone)
    for start, end in quiet_hours:
        quiet_start = current_time.replace(hour=start[0], minute=start[1], second=0, microsecond=0)
        quiet_end = current_time.replace(hour=end[0], minute=end[1], second=59, microsecond=0)
        if quiet_start <= current_time <= quiet_end:
            return False
    # Then continue process send to Telegram
    current_retry = 0
    while (current_retry <= max_retry):
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} Telegram: Retry #{current_retry}")
            time.sleep(5)   # Wait a second to stabilize the server
        try:
            # # OPTION 1 - URL encode the message string
            # encoded_message = urllib.parse.quote(message_string)
            # # Construct the URL
            # url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={ID}&text={encoded_message}"

            # OPTION 2 - KEEP raw message
            url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={ID}&text={message_string}"
            # Send the request
            response = requests.post(url, timeout=5)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:            
            print(date_time, f"ERROR send Telegram: \n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(date_time, "TELEGRAM: Max retries reached.")
                return False
        except Exception as err:
            print(f"Telegram other error occurred: {err}")      # Handle any other errors
            current_retry += 1

# Function Setup Socket to listen message from other services =======================
def setup_socket(host, port):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    try:
        server_socket.bind((host, port))
        server_socket.listen()
        print(f"{date_time} Server listening on {host}:{port}")
        return server_socket
    except OSError as e:
        print(f"{date_time} Error binding to {host}:{port} - {e}")
        server_socket.close()
        raise

# Function check and close Chrome process
def close_chrome():
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    try:        
        # List all running processes
        result = subprocess.run(['tasklist'], stdout=subprocess.PIPE, text=True)
        # Check if 'chrome.exe' is in the list of running processes
        if 'chrome.exe' in result.stdout:
            # Terminate all 'chrome.exe' processes
            os.system("taskkill /f /im chrome.exe")
            print(date_time, "Closed existing Chrome processes.")
        else:
            print(date_time, "No existing Chrome processes found.")
        time.sleep(1)
    except Exception as e:
        print(f"{date_time}: Error closing Chrome processes: {e}")

# Function to setup and keep the Chrome session
def setup_chrome():
    try:
        close_chrome()  # Close any Chrome process
        time.sleep(3)   # Waiting for system to stable
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(date_time, 'Setting up the Chrome process')

        # Set up Selenium WebDriver for Google Chrome
        chrome_options = Options()
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--log-level=3")    # Suppress logs
        chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
        # chrome_options.add_argument("--headless")     # Run in headless mode
        service = Service('')  # Update with your path to chromedriver
        driver = webdriver.Chrome(service=service, options=chrome_options)
        # Maximize the browser window
        driver.maximize_window()

        driver.get("https://goonus.io/signin")
        # Perform login (update with actual login steps)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("0987115281")
        time.sleep(1)
        password_field = driver.find_element(By.ID, "password")
        password_field.send_keys("Trien825590@OU")
        time.sleep(1)
        password_field.send_keys(Keys.RETURN)  # Simulate pressing Enter key
        print("\t\tLog-in Submitted")
        # Wait for login to complete
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "el-popover__reference-wrapper")))
        # Navigate to the desired URL
        print("\t\tNavigate to Chart page")
        driver.get("https://goonus.io/futures/BTC_VNDC")

        # Wait for the page to load
        # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "chart-gui-wrapper")))
        time.sleep(20)
        return driver

    except Exception as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time} Error setup Chrome: {e}")
        driver.quit()
        raise


# Function extract the token from Chrome cookies
def get_token(driver, refresh=True):
    global access_token
    global new_token
    global expires_token
    
    # Create the New
    new_token = None
    retry = 0
    while new_token is None:
        if retry > 0:
            time.sleep(60)
            print("\t\tRetry Getting the Token", retry)
        try:
            # Refresh the current page
            if refresh == True:
                driver.refresh()
                print('\t\tRefresh the browser')
                time.sleep(20)
            # Extract token from cookies
            cookies = driver.get_cookies()
            # Close the Selenium WebDriver
            # driver.quit()
            for cookie in cookies:
                if cookie['name'] == 'access-token-onus-pro':
                    print('\t\tGet access-token-onus-pro')
                    new_token = cookie['value']
                    expires = cookie['expiry']
                    expires_token = datetime.fromtimestamp(int(expires), tz=pytz.utc).astimezone(timezone)
                    break
                elif cookie['name'] == 'onuspro-access-token':
                    # print('\t\tGet onuspro-access-token')
                    new_token = cookie['value']
                    expires = cookie['expiry']
                    expires_token = datetime.fromtimestamp(int(expires), tz=pytz.utc).astimezone(timezone)

            # if new_token == access_token:
            #     print("\t\tKeep same Token:", access_token[-10:])
            if new_token != access_token:
                access_token = new_token
                print("\t\tNew Token:", access_token[-10:])
                print("\t\tToken expire at:", expires_token)
            return access_token, expires_token
        except Exception as e:
            print(e)
            retry += 1

# Get Account Balance
def get_balance():
    global access_token
    global new_token
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/account"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "Host": "api-pro.goonus.io"
    }
    try:
        # Make the POST request with headers
        response = requests.get(url, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        index = 0
        # Collect the VNDC value
        while(response_data['assets'][index]['asset']!='VNDC'):
            index +=1
        # print(response_data['assets'][index])
        
        # Tổng tài sản
        walletBalance = float(response_data['assets'][index]['walletBalance'])
        # print("Total Balance:", walletBalance)
        # Tài sản khả dụng để order
        availableBalance = float(response_data['assets'][index]['availableBalance'])
        # print("Available:",availableBalance)
        # Tài sản đang ký quỹ
        isolatedPositionMargin = float(response_data['assets'][index]['isolatedPositionMargin'])
        # print("Isolated:", isolatedPositionMargin)
        # Tài sản đang khóa (ký quỹ + đang order)
        orderMargin = float(response_data['assets'][index]['orderMargin'])
        orderFee = float(response_data['assets'][index]['orderFee'])
        lockedBalance = isolatedPositionMargin + orderMargin + orderFee
        # print("Locked:", lockedBalance)
        return walletBalance,availableBalance,lockedBalance

    except requests.RequestException as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time}: Error Get balance {e}")
        new_token = None
        return None,None,None

# Get Open Position list
def get_position():
    global access_token
    global new_token
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/positions?status=OPEN"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "Host": "api-pro.goonus.io"
    }
    try:
        # Make the POST request with headers
        response = requests.get(url, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        list_position = []
        for i in range(len(response_data)):
            # print(response_data[i]["symbol"])
            list_position.append(response_data[i]["symbol"])        
        return list_position
    except requests.RequestException as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time}: Error Get Position {e}")
        new_token = None
        return []

# Set Leverage
def set_leverage(coin_name, leverage_number):
    global access_token
    global new_token
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/leverage"
    data = {"symbol": coin_name, "leverage": leverage_number}
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    try:
        # Make the POST request with headers
        response = requests.post(url, json=data, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['userId']=='486e359b-20c4-4d32-bf9a-483d5c8ce3f8'):
            print(f'\t\tLeverage set for {coin_name} at x{leverage_number}')
        # if response_data['userId']

    except requests.RequestException as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time}: Error set Leverage {e}")
        new_token = None

# Place Strategic Order
def set_strategy(coin_name, price, amount, profit, loss):
    global access_token
    global new_token

    url = "https://api-pro.goonus.io/perpetual/v1/strategyOrder"
    payload = { "orders": [
        {
        "symbol": coin_name,
        "side": "BUY",
        "type": "MARKET",
        "positionSide": "BOTH",
        "price": price,
        "size": amount,
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "workingType": "MARK_PRICE",
        "stopPrice": "0",
        "reduceOnly": False,
        "timeInForce": "GTC",
        "closePosition": False
        },
        {
        "type": "TAKE_PROFIT",
        "price": 0,
        "stopPrice": profit,
        "workingType": "CONTRACT_PRICE",
        "symbol": coin_name,
        "side": "SELL",
        "size": amount,
        "positionSide": "BOTH",
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "reduceOnly": True,
        "closePosition": False
        },
        {
        "type": "STOP",
        "price": 0,
        "stopPrice": loss,
        "workingType": "CONTRACT_PRICE",
        "symbol": coin_name,
        "side": "SELL",
        "size": amount,
        "positionSide": "BOTH",
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "reduceOnly": True,
        "closePosition": False
        }
    ]}
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    # print(payload)
    try:
        # Make the POST request with headers
        response = requests.post(url, json=payload, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['status'] == 'WORKING'):
            print(f'Successfull place Strategic order {coin_name} ')
            return True
        else:
            print(f'Error: {response_data}')
            return False
    except requests.RequestException as e:
        print(f"{e}")
        new_token = None
        return False

# Run the main function =======================================================
if __name__ == "__main__":
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    message = f"{date_time} Gamma started"
    print(message)
    send_telegram(message)

    # Process Login and get the access token ==================================
    chrome = setup_chrome()
    get_token(chrome, refresh=False)

    # Config for Server socket ================================================
    server_socket = setup_socket('localhost', 65432)
    input_channel = [server_socket]
    
    try:        
        # Infinite loop to execute
        while True:
            # Monitors the sockets in the input_channel list for readability.
            # It returns three lists: readable, writable, and exceptional.
            # Here, we only care about readable with Timeout 1 second
            readable, _, _ = select.select(input_channel, [], [], 1)
            # Iterates over the sockets that are ready for reading
            for signal in readable:
                if signal is server_socket:
                    conn, addr = server_socket.accept()
                    input_channel.append(conn)
                    # print(f"Accepted connection from {addr}")
                else:
                    # Reads up to 1024 bytes of data from the socket.
                    data = signal.recv(1024)
                    # Checks if data was received
                    if data:
                        order_info = json.loads(data.decode('utf-8'))
                        
                        # START PROCESS =================================================
                        # {"coin_name": "BTCVNDC","order_price": 123.1}
                        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                        print(date_time,'RECEIVED -->', order_info)
                        
                        # Check if coin is allowed & avoid duplicate order
                        exclude_list = get_position()
                        print(f"\t\tCurrent position: {exclude_list}")
                        # Double check token
                        if (access_token is None) or (new_token is None):
                            print('\t\tNeed to renew Token')
                            get_token(chrome, refresh=True)

                        if  (order_info['coin_name'] in tuple_coin) and\
                            (order_info['coin_name'] not in exclude_list) and\
                            (time.time() - last_order > 300):
                            
                            print('\t\tPlacing new order')
                            # Get current available balance
                            total,available,locked = get_balance()
                            print(f"\t\tGetting available balance: {available}")
                            if total is not None:
                                # print(total,available,locked)
                                # Process choose the leverage --> variance but default x5
                                leverage = 5
                                set_leverage(order_info['coin_name'], leverage)
                                # Choose the amount VNDC of investment
                                # Idea: 20% of Total balance < availabe balane
                                # Profit 2.5% , Loss 10%
                                order_amount = total/20
                                order_price = order_info['order_price']
                                amount = round((leverage * order_amount)/order_price , 4)
                                profit = round((order_price + (2*order_price/leverage/100)), 3)
                                loss = round((order_price - (5.5*order_price/leverage/100)), 3)
                                if order_amount < available:
                                    # Place order
                                    # print("Order Amount:\t",order_amount)
                                    # print("Order Price:\t", order_price)
                                    # print("Amount:\t",amount)
                                    # print("Profit:\t", profit)
                                    # print("Loss:\t",loss)
                                    if set_strategy(order_info['coin_name'], order_price, amount, profit, loss):
                                        send_telegram(f"\t\tDone Order {order_info['coin_name']}")
                                        last_order = time.time()
                                else:
                                    print(f"\t\tOrder {order_info['coin_name']}: Balance not sufficient")

                    else:
                        # Removes the socket from the input_channel list.
                        input_channel.remove(signal)
                        signal.close()
            
            # Check point every 5 min
            if (time.time() - check_point > 120 ):
                check_point = time.time()
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                # print(date_time, "Process checkpoint")
                
                try:
                    total,available,lock = get_balance()
                    print(f"\t\tTotal: {round(total,0)}\tAvailable: {round(available,0)}")
                    get_token(chrome, refresh=False)
                except Exception as e:
                    print(f"Error Checkpoint: {e}")
                    get_token(chrome, refresh=True)
                    # Will add more condition to double check here !!!

                if new_token == None:
                    get_token(chrome, refresh=True)


    except KeyboardInterrupt:
        print("Server is shutting down...")
    finally:
        server_socket.close()    

    print(f"{date_time}: Stop Processing")
    # set_leverage("BTCVNDC", 5)


2024-12-08 23:15:15 Gamma started
2024-12-08 23:15:15 No existing Chrome processes found.
2024-12-08 23:15:19 Setting up the Chrome process
		Log-in Submitted
		Navigate to Chart page
		Get access-token-onus-pro
		New Token: Hk_zZslFWA
		Token expire at: 2025-12-08 23:16:34+07:00
2024-12-08 23:16:52 Server listening on localhost:65432
		Total: 523328.0	Available: 523328.0
		Get access-token-onus-pro
		Keep same Token: Hk_zZslFWA
		Total: 523328.0	Available: 523328.0
		Get access-token-onus-pro
		Keep same Token: Hk_zZslFWA
		Total: 523328.0	Available: 523328.0
		Get access-token-onus-pro
		Keep same Token: Hk_zZslFWA
Server is shutting down...
2024-12-08 23:21:17: Stop Processing
